# 키워드 검색 - Lexical Search


## 사전 준비


워크샵에서 사용할 데이터 파일을 읽어 판다스 데이터프레임으로 저장합니다.


In [1]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈


데이터의 스키마와 레코드 수를 확인합니다.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1000 non-null   object 
 1   genre       1000 non-null   object 
 2   year        1000 non-null   int64  
 3   date        1000 non-null   float64
 4   rating      1000 non-null   float64
 5   vote_count  1000 non-null   int64  
 6   plot        1000 non-null   object 
 7   main_act    1000 non-null   object 
 8   supp_act    1000 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 70.4+ KB


### OpenSearch 도메인에 연결하기


먼저 utils 디렉토리에 있는 공용모듈을 사용하기 위한 준비를 합니다.


In [3]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

이 코드를 통해 OpenSearch 도메인에 연결하는 데 필요한 자격 증명과 엔드포인트 주소를 가져올 수 있습니다. 이 정보를 사용하여 OpenSearch 클라이언트를 초기화하고 인덱싱, 검색 등의 작업을 수행할 수 있습니다.


In [4]:
from opensearchpy import OpenSearch
import json

# OpenSearch 연결 설정
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

aos_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False,
)

연결이 잘 되었는지 확인하기 위해 간단한 요청을 보내보겠습니다. analyze 엔드포인트를 사용하여 nori 분석기로 간단한 한국어를 분석해 봅니다.


In [5]:
request_body = {
    "analyzer": "nori",
    "text": "OpenSearch 워크샵에 오신 고객 여러분 환영합니다."
}

# Send the request to the _analyze endpoint
response = aos_client.indices.analyze(body=request_body)

# Print the response
print(json.dumps(response, indent=4, ensure_ascii=False))

{
    "tokens": [
        {
            "token": "opensearch",
            "start_offset": 0,
            "end_offset": 10,
            "type": "word",
            "position": 0
        },
        {
            "token": "워크",
            "start_offset": 11,
            "end_offset": 13,
            "type": "word",
            "position": 1
        },
        {
            "token": "샵",
            "start_offset": 13,
            "end_offset": 14,
            "type": "word",
            "position": 2
        },
        {
            "token": "오",
            "start_offset": 16,
            "end_offset": 17,
            "type": "word",
            "position": 4
        },
        {
            "token": "고객",
            "start_offset": 19,
            "end_offset": 21,
            "type": "word",
            "position": 7
        },
        {
            "token": "여러분",
            "start_offset": 22,
            "end_offset": 25,
            "type": "word",
            "position": 8
   

### 인덱스 생성


이제 인덱스를 생성합니다. 인덱스의 이름은 movies_lexical 로 하고 맵핑 정보는 다음과 같이 지정합니다.|


In [6]:
index_name = "movie_lexical"

movie_lexical = {
    "settings": {
        "number_of_replicas": 0,
        "number_of_shards": 1,
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {
                "type": "float"
            },
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {
                "type": "long"
            },
            "year": {
                "type": "long"
            },
        }
    },
}

위의 맵핑 정보로 인덱스를 생성합니다.


In [8]:
aos_client.indices.delete(index=index_name)
aos_client.indices.create(index=index_name, body=movie_lexical)
aos_client.indices.get(index=index_name)

{'movie_lexical': {'aliases': {},
  'mappings': {'properties': {'date': {'type': 'float'},
    'genre': {'type': 'text'},
    'main_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'plot': {'type': 'text'},
    'rating': {'type': 'float'},
    'supp_act': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'title': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vote_count': {'type': 'long'},
    'year': {'type': 'long'}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '1',
    'provided_name': 'movie_lexical',
    'max_result_window': '15000',
    'creation_date': '1724140613209',
    'analysis': {'analyzer': {'analysis-nori': {'type': 'nori',
       'stopwords': '_korean_'}}},
    'number_of_replicas': '0',
    'uuid': 'xnhVmzBGSGCKBTj6WyJYaA',
    'version': {'created': '136347827'}}}}}

### 데이터 인제스트


생성된 인덱스에 데이터를 인제스트합니다. 여기서는 opensearchpy 패키지에서 제공하는 parallel_bulk를 사용하여 빠르게 데이터를 인제스트합니다.


In [9]:
from opensearchpy import helpers

# Pandas DataFrame을 JSON 형식의 문자열로 변환
json_data = df.to_json(orient="records", lines=True)
# JSON 문자열을 개별 JSON 객체로 분할하고, 마지막 빈 줄을 제거
docs = json_data.split("\n")[:-1]  # To remove the last empty line


# JSON 객체를 OpenSearch에 업로드할 수 있는 형식으로 변환
def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
# 병렬로 벌크 업로드를 수행
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=20, thread_count=4, queue_size=4
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터가 바로 반영되도록 인덱스를 Refresh하고 인제스트가 잘 되었는지 확인하기 위해 도큐먼트의 총 합을 count로 확인합니다.


In [10]:
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 1000, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


이제 키워드 검색을 수행할 모든 준비가 끝났습니다.


## 키워드 검색 결과 확인하기


키워드 검색을 위한 헬퍼 함수를 생성합니다. 여기서는 상위 10개의 결과만을 확인합니다.


In [11]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["title", "plot", "genre", "main_act"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=["_score", "title", "plot", "genre", "rating", "main_act"],
    )
    display(query_result_df)

자연어로 검색결과를 확인해봅니다.


In [12]:
keyword_search("지구의 영웅들이 힘을 합쳐 우주의 악당을 물리친다")

,_score,title,plot,genre,rating,main_act
0,11.633025,캡틴 마블,"1995년, 공군 파일럿 시절의 기억을 잃고 크리족 전사로 살아가던 캐럴 댄버스(브...",액션|모험|SF,6.75,브리 라슨|사무엘 L. 잭슨|벤 멘델슨|주드 로
1,10.987896,스타워즈: 깨어난 포스,새로운 전설을 그려나가게 될 포스의 선택을 받은 ‘레이’ 와 포스의 기운을 모아 정...,액션|모험|판타지,7.75,데이지 리들리|존 보예가|오스카 아이삭|아담 드라이버|그웬돌린 크리스티|도널 글리슨...
2,9.114580,샤잠!,"솔로몬의 지혜, 헤라클레스의 힘, 아틀라스의 체력, 제우스의 권능, 아킬레스의 용기...",액션|판타지|SF,5.82,제커리 레비|애셔 앤젤
3,9.070051,트랜스포머: 패자의 역습,"샘 윗윅키(샤이아 라보프)가 오토봇과 디셉티콘, 두 로봇 진영간의 치열한 싸움에서 ...",SF|액션|모험,8.11,샤이아 라보프|메간 폭스|휴고 위빙|조쉬 더하멜|존 터투로
4,8.260200,범죄와의 전쟁 : 나쁜놈들 전성시대,"비리 세관 공무원 최익현, 보스 최형배를 만나다! 1982년 부산. 해고될 위기에 ...",범죄|드라마,8.63,최민식|하정우
5,7.808550,저스티스 리그,인류의 수호자인 슈퍼맨이 사라진 틈을 노리고 ‘마더박스’를 차지하기 위해 빌런 스테...,액션|모험|판타지|SF,7.70,벤 애플렉|갤 가돗|제이슨 모모아|레이 피셔|에즈라 밀러|헨리 카빌
6,7.562352,드래곤볼 에볼루션,우주 각지에 흩어진 7개의 구슬을 모두 모으면 엄청난 힘을 가질 수 있는 전설 속의...,액션|모험|판타지|SF|스릴러,3.21,저스틴 채트윈|에미 로섬|주윤발|제임스 마스터스|박준형|제이미 정
7,7.541350,하루,전쟁의 성자라 불리는 의사 ‘준영’(김명민)은 딸의 생일 날 약속 장소로 향하던 중...,스릴러,7.05,김명민|변요한|유재명
8,6.732153,맨 인 블랙 3,알 수 없는 사건으로 현실이 뒤바뀌고 외계인의 공격으로 위험에 빠진 지구. 게다가 ...,액션|코미디|SF,8.18,윌 스미스|토미 리 존스|조슈 브롤린
9,6.576585,오블리비언,"외계인의 침공이 있었던 지구 최후의 날 이후, 모두가 떠나버린 지구의 마지막 정찰병...",액션|SF,8.25,톰 크루즈|모건 프리먼|올가 쿠릴렌코
